In [4]:
import numpy as np
import pandas as pd
import pickle
from preprocessing_functions import *
from predict_function import *
import praw
import re
import joblib

fastText_model = joblib.load("fast_text_model.pkl")
classifier = joblib.load("classifier.pkl")
scalar = joblib.load("scaler.pkl")


reddit = praw.Reddit(client_id="z8UhRRiFnEVZ8Q",
                     client_secret="bV7OxwG-VKjdyKxcuUihnV1lNPg",
                     password="qazwsx123",
                     username="lshreyasharmal",
                     user_agent="bot1 user agent")
final_features = [u'author', u'comments', u'is_original_content',
       u'is_reddit_media_domain', u'is_video', u'over_18', u'permalink',
       u'secure_media', u'selftext', u'send_replies', u'title', u'url',
       u'edited', u'num_comments', u'num_duplicates', u'subreddit_subscribers',
       u'ups', u'upvote_ratio']
numerical_cols = [u'edited', u'num_comments',u'num_duplicates',u'subreddit_subscribers', u'ups', u'upvote_ratio']
flairs_mapping = {1:'AskIndia',2:'Business/Finance',3:'Food',4:'Non-Political',5:'Photography',6:'Policy/Economy',7:'Politics',8:'Scheduled',9:'Science/Technology',10:'Sports',0:'Others'}
    

def get_feature_dict(s):
    to_dict = vars(s)
    y_true = s.link_flair_text
    to_dict['upvote_ratio'] = s.upvote_ratio
    author = str(s.author).split("'")
    to_dict['author'] = author[len(author)-2]
    comments = []
    s.comments.replace_more(limit=None)
    for comment in s.comments.list():
        comments.append(comment.body)
    to_dict['comments'] = comments
    sub_dict = {field:to_dict[field] for field in final_features}
    return y_true,sub_dict


def get_flair(code):
    s = reddit.submission(code)
    y_true,sub_dict = get_feature_dict(s)
    new_data = hanlde_bool_and_tokenize([sub_dict])
    new_data = pd.DataFrame.from_records(new_data)
    numerical_data = new_data[numerical_cols]
    numeric_X  = scalar.transform(numerical_data)
    numeric_X = pd.DataFrame(numeric_X, index=numerical_data.index, columns=numerical_data.columns)
    X = new_data.drop(numerical_cols,axis=1)
    X_final = X.join(numeric_X)
    obj_cols = X_final.select_dtypes(include=[object]).columns
    temp_X = X_final
    for obj_col in obj_cols:
        temp_X = get_obj_column(obj_col,temp_X,fastText_model)
    final_data = get_given_features_from_data(final_features,temp_X)
    return flairs_mapping[classifier.predict(final_data)[0]]
    
def predict_flair_from_url(url_link):
    code = ""
    try:
        url_link_formatted = re.sub(r'https://www.reddit.com/','',url_link)
        url_tokens = url_link_formatted.split("/")
        if url_tokens[0]!="r" or url_tokens[1]!="india":
            return "Error: Incorrect Link"
        for i in range(len(url_tokens)):
            if(url_tokens[i]=='comments'):
                code = url_tokens[i+1]
    except:
        return "Error Occurred!"
    if(code!=""):
        return get_flair(code)
    else:
        return "post code not found"
    

In [5]:
predict_flair_from_url("https://www.reddit.com/r/india/comments/daq2vy/weekly_mental_health_support_thread_september_29/")

'Non-Political'